In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load data
df = pd.read_csv('Aggregated_Cleaned_Merged_Dataset_No_FIPS.csv')
df.drop(["State.Minimum.Wage", "Federal.Minimum.Wage","Effective.Minimum.Wage"],axis=1,inplace=True)
df.head()

,Year,State,State.Minimum.Wage.2020.Dollars,Federal.Minimum.Wage.2020.Dollars,Effective.Minimum.Wage.2020.Dollars,CPI.Average,Total Civilian Non-Institutional Population in State/Area,Total Civilian Labor Force in State/Area,Total Employment in State/Area,Total Unemployment in State/Area,Percent (%) of Labor Force Unemployed in State/Area
0,1976,Alabama,0.00,10.0,10.00,56.9,2.632667e+06,1.500260e+06,1.399782e+06,100477.583333,6.700000
1,1976,Alaska,12.73,10.0,12.73,56.9,2.399167e+05,1.642791e+05,1.518328e+05,12446.250000,7.566667
2,1976,Arizona,0.00,10.0,10.00,56.9,1.650917e+06,9.819918e+05,8.861592e+05,95832.583333,9.758333
3,1976,Arkansas,8.64,10.0,10.00,56.9,1.546583e+06,8.937798e+05,8.321712e+05,61608.666667,6.916667
4,1976,California,9.09,10.0,10.00,56.9,1.582375e+07,9.898940e+06,8.993283e+06,905657.250000,9.150000


In [3]:
!pip install interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 16.7 MB/s eta 0:00:00
  Created wheel for dash-cytoscape: filename=dash_cytoscape-1.0.2-py3-none-any.whl size=40

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


X = df[["Year", "State.Minimum.Wage.2020.Dollars", "Federal.Minimum.Wage.2020.Dollars", "CPI.Average","State",
        "Total Civilian Non-Institutional Population in State/Area", "Total Civilian Labor Force in State/Area"]]
X = pd.get_dummies(X, columns=["State"], drop_first=True).astype(float)

y = df["Percent (%) of Labor Force Unemployed in State/Area"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from interpret import show
from interpret.data import Marginal
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree
ebm = ExplainableBoostingRegressor(random_state=42, n_jobs=-1)
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(n_jobs=-1)

In [8]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

In [9]:
from interpret.perf import RegressionPerf

ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)